# **PucMinas TCC**: Especialização em Ciência de Dados e Big Data 
### Stanley Cruvinel 
#### Dados raspados da Fretes da Truckpad pelo AirFlow

## 1. Objetivo

Construir 2 Graphos de Origem Destino dos dados de Frete para saber quais as maiores demandas por Fretes.
* 1º para Municípios
* 2ºpara os Estados

## 2. Recurso utilizado será o The Spark Cluster com Pyspark

### 2.1.Concecção

Para conectar com o cluster do Spark, é criada uma SparkSession object com parâmetros:

+ **appName:** nome da aprlicação aprece [Spark Master Web UI](http://localhost:8080/);
+ **master:** URL do Spark Master URL, mesmo utilizado pelos Spark Workers;
+ **spark.executor.memory:** igual ao docker compose SPARK_WORKER_MEMORY config.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import networkx as nx
import pandas as pd
from pyvis.network import Network
#from IPython.core.display import display, HTML

In [2]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook-tcc_pucMinas").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "7G").\
        getOrCreate()

In [3]:
fretes = spark.read.csv(path="data/fretes.csv", sep=";", header=True)

In [4]:
fretes.count()

8751

In [5]:
fretes.show(10)

+---+----------+---------+-------------------+------------------+-------------+---+----+----------+-------------+-------------+--------------------+--------------------+----+---+---+
|_c0|UF_Destino|UF_origem|             Origem|           Destino|           KM|PED|Pton|     Preco|      Produto|      Especie|           Veiculo_1|           Veiculo_2| Ano|Mes|Dia|
+---+----------+---------+-------------------+------------------+-------------+---+----+----------+-------------+-------------+--------------------+--------------------+----+---+---+
|  0|        SP|       PR|          Arapongas|   Itaquaquecetuba|          616|  0|   0|      null|Não informado|Não informado|             'Truck'|          'Bi-Truck'|2021|  2|  1|
|  1|        SC|       CE|          Fortaleza|Balneário Camboriú|         3520|  0|   0|   150,00 |Não informado|Não informado|'Carreta LS Truca...|'Carreta LS Traça...|2021|  2|  1|
|  2|        RJ|       RJ|     Rio de Janeiro|    Rio de Janeiro|Não informado|  0|  

**Um dataframe cidades e outro com UF** 

In [6]:
edges_fretes_sc_df = fretes.selectExpr("upper(concat(UF_origem,' - ',Origem)) as source",
                            "upper(concat(UF_Destino,' - ',Destino)) as target",
                            "KM as km",
                            "regexp_replace(Preco,'([ˆ.]|[^,]*$)|[ˆ,]','') as preco").\
                selectExpr("*","format_number(preco/km,4) as preco_km").\
                where("not isnull(preco_km) and Pton = 0 and PED = 0")

In [7]:
edges_fretesuf_sc_df = fretes.selectExpr("upper(UF_origem) as source",
                            "upper(UF_Destino) as target",
                            "KM as km",
                            "regexp_replace(Preco,'([ˆ.]|[^,]*$)|[ˆ,]','') as preco").\
                selectExpr("*","format_number(preco/km,4) as preco_km").\
                where("not isnull(preco_km) and Pton = 0 and PED = 0")

In [8]:
edges_fretes_sc_df.show(10, False)

+--------------------------+-------------------------+----+-----+--------+
|source                    |target                   |km  |preco|preco_km|
+--------------------------+-------------------------+----+-----+--------+
|CE - FORTALEZA            |SC - BALNEÁRIO CAMBORIÚ  |3520| 150 |0.0426  |
|SP - SÃO JOSÉ DOS CAMPOS  |MG - DIVINÓPOLIS         |537 | 300 |0.5587  |
|PA - PARAGOMINAS          |PA - TAILÂNDIA           |265 | 3   |0.0113  |
|SP - RIO CLARO            |SP - RIBEIRÃO PRETO      |162 | 25  |0.1543  |
|RS - NOVA ALVORADA        |SP - VINHEDO             |1073| 2000|1.8639  |
|TO - GURUPI               |GO - APARECIDA DE GOIÂNIA|625 | 1000|1.6000  |
|SP - SÃO PAULO            |SP - BAURU               |327 | 200 |0.6116  |
|SP - SÃO BERNARDO DO CAMPO|SP - SÃO PAULO           |21  | 318 |15.1429 |
|SP - SANTOS               |SP - UBATUBA             |282 | 500 |1.7730  |
|RJ - RIO DE JANEIRO       |MG - COIMBRA             |341 | 750 |2.1994  |
+------------------------

In [9]:
edges_fretesuf_sc_df.show(10, False)

+------+------+----+-----+--------+
|source|target|km  |preco|preco_km|
+------+------+----+-----+--------+
|CE    |SC    |3520| 150 |0.0426  |
|SP    |MG    |537 | 300 |0.5587  |
|PA    |PA    |265 | 3   |0.0113  |
|SP    |SP    |162 | 25  |0.1543  |
|RS    |SP    |1073| 2000|1.8639  |
|TO    |GO    |625 | 1000|1.6000  |
|SP    |SP    |327 | 200 |0.6116  |
|SP    |SP    |21  | 318 |15.1429 |
|SP    |SP    |282 | 500 |1.7730  |
|RJ    |MG    |341 | 750 |2.1994  |
+------+------+----+-----+--------+
only showing top 10 rows



In [10]:
nodes_fretes_sc_df = edges_fretes_sc_df.\
                     selectExpr("source as label").\
                     union(edges_fretes_sc_df.\
                     selectExpr("target as label")).\
                     groupBy("label").count().sort(desc("count")).\
                     selectExpr("label","count as size", "concat('Arestas:',\
                                cast(count as string), ' ',label) as title").\
                     where("size > 4")

In [11]:
nodes_fretesuf_sc_df = edges_fretesuf_sc_df.\
                     selectExpr("source as label").\
                     union(edges_fretesuf_sc_df.\
                     selectExpr("target as label")).\
                     groupBy("label").count().sort(desc("count")).\
                     selectExpr("label","count as size", "concat('Arestas:',\
                                cast(count as string), ' ',label) as title").\
                     where("size > 4")

In [12]:
nodes_fretes_sc_df.show(10,False)

+----------------------------+----+---------------------------------------+
|label                       |size|title                                  |
+----------------------------+----+---------------------------------------+
|BA - SALVADOR               |225 |Arestas:225 BA - SALVADOR              |
|RJ - RIO DE JANEIRO         |80  |Arestas:80 RJ - RIO DE JANEIRO         |
|RS - TRÊS PASSOS            |60  |Arestas:60 RS - TRÊS PASSOS            |
|SP - SÃO PAULO              |44  |Arestas:44 SP - SÃO PAULO              |
|PE - JABOATÃO DOS GUARARAPES|23  |Arestas:23 PE - JABOATÃO DOS GUARARAPES|
|PI - TERESINA               |22  |Arestas:22 PI - TERESINA               |
|PB - BAYEUX                 |21  |Arestas:21 PB - BAYEUX                 |
|PA - MARITUBA               |21  |Arestas:21 PA - MARITUBA               |
|CE - EUSÉBIO                |21  |Arestas:21 CE - EUSÉBIO                |
|BA - VITÓRIA DA CONQUISTA   |21  |Arestas:21 BA - VITÓRIA DA CONQUISTA   |
+-----------

In [13]:
nodes_fretesuf_sc_df.show(30,False)

+-----+----+--------------+
|label|size|title         |
+-----+----+--------------+
|BA   |269 |Arestas:269 BA|
|SP   |195 |Arestas:195 SP|
|RS   |139 |Arestas:139 RS|
|RJ   |94  |Arestas:94 RJ |
|PE   |60  |Arestas:60 PE |
|MG   |55  |Arestas:55 MG |
|PA   |42  |Arestas:42 PA |
|CE   |37  |Arestas:37 CE |
|PR   |30  |Arestas:30 PR |
|PI   |24  |Arestas:24 PI |
|PB   |24  |Arestas:24 PB |
|MT   |20  |Arestas:20 MT |
|SC   |17  |Arestas:17 SC |
|GO   |15  |Arestas:15 GO |
|MA   |15  |Arestas:15 MA |
|SE   |14  |Arestas:14 SE |
|ES   |7   |Arestas:7 ES  |
|AL   |6   |Arestas:6 AL  |
|MS   |6   |Arestas:6 MS  |
+-----+----+--------------+



In [14]:
nodes_fretes = nodes_fretes_sc_df.toPandas()
edges_fretes = edges_fretes_sc_df.toPandas()

In [15]:
edges_fretes

,source,target,km,preco,preco_km
0,CE - FORTALEZA,SC - BALNEÁRIO CAMBORIÚ,3520,150,0.0426
1,SP - SÃO JOSÉ DOS CAMPOS,MG - DIVINÓPOLIS,537,300,0.5587
2,PA - PARAGOMINAS,PA - TAILÂNDIA,265,3,0.0113
3,SP - RIO CLARO,SP - RIBEIRÃO PRETO,162,25,0.1543
4,RS - NOVA ALVORADA,SP - VINHEDO,1073,2000,1.8639
...,...,...,...,...,...
537,SP - SÃO ROQUE,SP - MAIRINQUE,6193,60,0.0097
538,SP - SÃO ROQUE,SP - MAIRINQUE,6193,60,0.0097
539,SP - SÃO ROQUE,SP - MAIRINQUE,6193,60,0.0097
540,PA - SANTARÉM,PE - PETROLINA,2346,2000,0.8525


In [16]:
G = nx.from_pandas_edgelist(edges_fretes, 'source', 'target', True, nx.DiGraph())

In [17]:
nx.set_node_attributes(G, nodes_fretes.set_index('label').to_dict('index'))

In [18]:
net = Network(height="1500px", width="100%", bgcolor="#222222", font_color="white",directed=True, notebook=True, heading='Fretes')
net.from_nx(G)

In [19]:
options = '''
var options = {
  "physics": {
    "forceAtlas2Based": {
      "springLength": 100
    },
    "minVelocity": 0.75,
    "solver": "forceAtlas2Based"
  }
}'''

In [20]:
net.set_edge_smooth('dynamic')
net.set_options(options)
#net.show_buttons(filter_=['physics'])
net.show('view/tcc_pucminas_cidades.html')
#display(HTML('view/tcc_puc_cidades.html'))

In [21]:
nodes_fretesuf = nodes_fretesuf_sc_df.toPandas()
edges_fretesuf = edges_fretesuf_sc_df.toPandas()

In [22]:
Guf = nx.from_pandas_edgelist(edges_fretesuf, 'source', 'target', True, nx.DiGraph())

In [23]:
nx.set_node_attributes(Guf, nodes_fretesuf.set_index('label').to_dict('index'))

In [24]:
netuf = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white",directed=True, notebook=True, heading='Fretes')
netuf.from_nx(Guf)

In [25]:
options2 = '''
var options = {
  "physics": {
    "forceAtlas2Based": {
      "springLength": 100
    },
    "minVelocity": 0.75,
    "solver": "forceAtlas2Based"
  }
}'''

In [26]:
netuf.set_edge_smooth('dynamic')
netuf.set_options(options2)
netuf.show('view/pucminas_tcc_fretesuf.html')
#display(HTML(''view/pucminas_tcc_fretesuf.html''))